Import required modules

In [ ]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
import json
import patient_data

Access the folder path for the cancer and the non-cancer images

In [22]:
all_paths = json.loads(open("./paths.json").read())

personal_path = all_paths['personal_path']
non_cancerous_path = personal_path + all_paths['non_cancerous_path']
cancerous_path = personal_path + all_paths['cancerous_path']

Load in all the DICOM files and preprocess/label images

In [23]:
# Using the patient_data data structure, load in all the patient data and save it in a dictionary with the folder name as the key
def load_all_patients(path):
    patients = {}
    folder = os.listdir(path)
    for name in folder:
        patients[name] = patient_data.Patient(os.path.join(path, name))
    return patients

def label_cancerous(patients):
    for patient in patients.values():
        patient.label_imgs()

nc_patients = load_all_patients(non_cancerous_path)
c_patients = load_all_patients(cancerous_path)
label_cancerous(c_patients)

Save images from dicoms as JPEGS for visualization

In [24]:
# def convert_to_jpg(patients, output_folder):
#     for patient in patients:
#         path = os.path.join(output_folder, patient)
#         patients[patient].save_data_as_images(path)

# cip = personal_path + "/all_images/cancerous/"
# ncip = personal_path + "/all_images/non_cancerous/"

# convert_to_jpg(c_patients, cip)
# convert_to_jpg(nc_patients, ncip)

Overlay annotations on CT images and produce JPEGS for visualization

In [25]:
# def save_overlays(patients, output_folder):
#     for patient in patients.keys():
#         path = os.path.join(output_folder, patient)
#         patients[patient].overlay_seg_on_ct(path)
#     return 

# cip = personal_path + "/all_images/cancerous/"
# ncip = personal_path + "/all_images/non_cancerous/"

# save_overlays(c_patients, cip)
# save_overlays(nc_patients, ncip)

Setting up train/test data

In [45]:
# # not sure if we need this
# # create a list for the merged data
# x = []
# y = []

# create a list for only the cancerous dataset data
x_c = []
y_c = []
# create a list for only the non-cancerous dataset data
x_nc = []
y_nc = []

for patient in c_patients.values():
    for i, img in enumerate(patient.ct.images):
        x_c.append(img)
        y_c.append(patient.labels[i])
        # # not sure if we need this
        # x.append(img)
        # y.append(patient.labels[i])

for patient in nc_patients.values():
    for i, img in enumerate(patient.ct.images):
        x_nc.append(img)
        y_nc.append(patient.labels[i])
        # # not sure if we need this
        # x.append(img)
        # y.append(patient.labels[i])

# # not sure if we need this
# # Shuffle the merged data
# combined = list(zip(x, y))
# np.random.shuffle(combined)
# x2, y2 = zip(*combined)

def generate_train_test():
    # to ensure equal distribution of non-cancer to cancer data, split the data before merging it
    x_train, x_test, y_train, y_test = train_test_split(x_c, y_c, test_size=0.2, random_state=42)
    x_train_add, x_test_add, y_train_add, y_test_add = train_test_split(x_nc, y_nc, test_size=0.2, random_state=42)
    x_train.extend(x_train_add) 
    x_test.extend(x_test_add) 
    y_train.extend(y_train_add) 
    y_test.extend(y_test_add) 

    # Convert lists to arrays
    x_train = np.array(x_train)
    x_test = np.array(x_test)
    y_train = np.array(y_train)
    y_test = np.array(y_test)

    return x_train, x_test, y_train, y_test

num_tests = 3
test_data = []
for i in range(num_tests):
    test_data.append(generate_train_test())

In [ ]:
# # double checking shape of the data

# print("X_train shape:", x_train.shape)
# print("X_test shape:", x_test.shape)
# print("y_train shape:", y_train.shape)
# print("y_test shape:", y_test.shape)

Check Class Distribution in Train and Test Sets

In [44]:
for i in test_data:
    # Assuming y_train and y_test are your labels for the train and test sets
    train_class_distribution = pd.Series(i[2]).value_counts(normalize=True)
    test_class_distribution = pd.Series(i[3]).value_counts(normalize=True)

    print("Class distribution in training set:")
    print(train_class_distribution)
    print("\nClass distribution in testing set:")
    print(test_class_distribution)

TypeError: 'function' object is not subscriptable

Train and test CNN model

Cross validation and bootstrapping